<a href="https://colab.research.google.com/github/WSD2019-teamD/data_reduction/blob/master/colab/similarity_LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

類似度に関するjsonを作るところまで

 TODO: 一ヶ月分でも類似度のソートが終わらない

In [0]:
from urllib import request 
import logging
from pathlib import Path
import numpy as np
import re
# import MeCab
import random
from gensim import corpora, models
import pandas as pd
from tqdm import tqdm

import pickle
import gensim

# 準備
データをCSVから読み込む

##colabの設定



### gensimをupdateしたらrestartが必要

In [0]:
# from google.colab import auth
# auth.authenticate_user()

In [10]:
!pip install gensim  --upgrade

     |████████████████████████████████| 24.2MB 40.1MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [1]:
pip list | grep gensim

gensim                   3.8.1      


### google dirveをマウント

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import os
os.chdir('/content/drive/My Drive/Colab_Notebooks/')
print(os.getcwd())

/content/drive/My Drive/Colab_Notebooks


In [0]:
from urllib import request 
import logging
from pathlib import Path
import numpy as np
import re
# import MeCab
import random
from gensim import corpora, models
import pandas as pd
from tqdm import tqdm

import pickle
import gensim

##コーパス、モデルをインポート

In [0]:
from smart_open import open
from gensim import models, corpora, similarities

corpus = corpora.MmCorpus("data/lda/cop.mm")
lda = models.ldamodel.LdaModel.load("data/lda/lda.model")
# d = corpora.Dictionary.load_from_text("data/lda/dict.txt")
# doc_index = similarities.docsim.MatrixSimilarity.load("datalda//sim")

##dfの準備

In [17]:
dir_path = os.curdir
print(dir_path)

files = [path for path in os.listdir(dir_path + '/data/article')]
print(files)

.
['2019-01-31_00:00:00-content.csv', '2018-12-31_00:00:00-content.csv', '2018-10-31_00:00:00-content.csv', '2018-09-30_00:00:00-content.csv', '2018-08-31_00:00:00-content.csv', '2018-07-31_00:00:00-content.csv', '2018-06-30_00:00:00-content.csv', '2018-05-31_00:00:00-content.csv', '2018-04-30_00:00:00-content.csv', '2018-03-31_00:00:00-content.csv', '2018-02-28_00:00:00-content.csv', 'model.tfidf', 'wordcloud.png']


In [0]:
df = pd.DataFrame()
for file in files[:1]:
    df_buf =  pd.read_csv('./data/article/' + file)
    df = pd.concat([df, df_buf], axis = 0,  ignore_index=True)

In [21]:
df.head(1)

,Unnamed: 0,id,article_id,title,created_at,likes_count,tags_str,url,sharing
0,0,30,b9c5eb649ada9224e928,CloudFunctionsからCloudStorageへ画像をアップロードする,2019-01-31 00:01:26,8,"CloudStorage,Firebase,FirebaseCloudFunctions",https://qiita.com/shim0mura/items/b9c5eb649ada...,"やる, てる, の, cloudfunctions, ある, ファイル, 画像, 読み込める..."


# 類似度

In [0]:
from gensim import similarities
doc_index = similarities.docsim.MatrixSimilarity(lda[corpus])

In [6]:
len(doc_index)

8225

In [7]:
c = corpus[0]
vec_lda = lda[c]
vec_lda

[(9, 0.08262868),
 (18, 0.07383547),
 (22, 0.22890331),
 (25, 0.4853964),
 (26, 0.10615211)]

In [0]:
s = doc_index.__getitem__(vec_lda)

In [0]:
s = sorted(enumerate(s), key=lambda t: t[1], reverse=True)

In [10]:
s[:15]

[(0, 1.0),
 (1627, 0.9368011),
 (959, 0.92413175),
 (4343, 0.920197),
 (4048, 0.9164375),
 (3682, 0.9060695),
 (1823, 0.89425117),
 (5441, 0.88342464),
 (5406, 0.8816956),
 (2548, 0.8772645),
 (3693, 0.87576747),
 (3215, 0.869621),
 (4370, 0.869621),
 (2075, 0.86572325),
 (8197, 0.8643372)]

In [0]:
def basic_info(num):
  if(num > len(df)):
    print("error")
    return
  print(df['url'][num])
  print(df['title'][num])
  print(df['tags_str'][num])
  print('like count: ' + str(df['likes_count'][num]))

In [0]:
def document_corpus(num):
  print('文書のトピックの関連')
  topics = sorted(lda.get_document_topics(corpus[0]), key=lambda t:t[1], reverse=True)
  basic_info(num)
  print('-' * 20) 
  # for t in topics[:5]:
  #   print("Topic # ",t[0])
  #   get_topic_words(t[0])
  #   print("\n")
    # print("{}: {}".format(t[0], t[1]))


類似している記事dfで返す

In [0]:
def similar_df(index, lda_model, corpus, similar_num,  debug=True):
  # drop_list = ["sharing", "Unnamed", "created_at"]
  drop_list = ["sharing", "created_at", "article_id", 	"likes_count", "tags_str","title", "url", "Unnamed: 0" ]
  vec_lda = lda_model[corpus[index]] #50zigenn
  s = doc_index.__getitem__(vec_lda) #記事の数
  s = sorted(enumerate(s), key=lambda t: t[1], reverse=True) #並び替え（ヤバい）
  df_json = pd.DataFrame()
  for doc_id, sim in s[1:similar_num+1]:
    if debug:
      print("similarity: " + str(sim))
      basic_info(doc_id)
      print("===========")
    # df_json = pd.concat([df_json, df.loc[doc_id]], axis=0, sort=True)
    series = df.loc[doc_id].copy()
    series["similarity"] = sim
    df_json = df_json.append(series)

  df_json_ed = df_json.drop(drop_list,  axis=1)
  return df_json_ed


In [0]:
def similar_json(index, lda_model, corpus, similar_num,   debug=True):
  df_json = similar_df(index, lda_model, corpus, similar_num,  debug=debug)
  return df_json.to_json(orient='records')

In [111]:
make_json(0, debug=False)

'[{"id":107780.0,"similarity":0.9368265867},{"id":108448.0,"similarity":0.9241193533},{"id":105064.0,"similarity":0.9202187061},{"id":105359.0,"similarity":0.9164627194}]'

In [0]:
def append_json_to_df(df_j,  lda_model, corpus,  similar_num):
  json_list = []
  for i in range(len(df_j)):
     json_list.append(similar_json(i,  lda_model, corpus, similar_num,  debug=False))
  print(len(json_list))
  df_j["json"] = json_list
  return df_j

    

In [113]:
append_json_to_df(df[:10].copy(), lda, corpus, 5)

10


,Unnamed: 0,id,article_id,title,created_at,likes_count,tags_str,url,sharing,json
0,0,30,b9c5eb649ada9224e928,CloudFunctionsからCloudStorageへ画像をアップロードする,2019-01-31 00:01:26,8,"CloudStorage,Firebase,FirebaseCloudFunctions",https://qiita.com/shim0mura/items/b9c5eb649ada...,"やる, てる, の, cloudfunctions, ある, ファイル, 画像, 読み込める...","[{""id"":107780.0,""similarity"":0.9368427396},{""i..."
1,1,29,06be5a8075e0c7924dbf,第二種スターリング数の指数型母関数表示とその応用,2019-01-31 00:03:56,1,"数学,組み合わせ",https://qiita.com/kaityo256/items/06be5a8075e0...,"スターリング数, stirling, Number, ご存知, 組み合わせ, 問題, 顔, ...","[{""id"":234065.0,""similarity"":0.901139915},{""id..."
2,2,28,5789ee7a724c6cb786a9,Oracle Cloud(OCI)のcliのinput/outputで使うjsonに慣れる,2019-01-31 00:09:57,3,"JSON,cli,oci,oraclecloud",https://qiita.com/feifo/items/5789ee7a724c6cb7...,"新規作成, tips, 追加, Oracle Cloud, infrastructure, ...","[{""id"":235553.0,""similarity"":0.8844355345},{""i..."
3,3,27,e9ff4d797a19c84231a1,[.NET] コードを見直したくなる「参照型」等価判定の思わぬ落とし穴（特殊編）,2019-01-31 00:10:46,1,"VB.Net,C#,.NET,tips,等価演算子",https://qiita.com/CodeOne/items/e9ff4d797a19c8...,"続く, 特殊, インターン, プール, 文字列, リテラル, インターン, プール, テーブ...","[{""id"":233407.0,""similarity"":0.9942728877},{""i..."
4,4,26,a31c2405a4a32d43bb24,【テンプレートリテラル】と【オブジェクトリテラル】を覚えたら「ちょっと」いい感じにコードが書けた！,2019-01-31 00:14:52,30,"JavaScript,es6",https://qiita.com/micropig3402/items/a31c2405a...,"概要, テンプレート, リテラル, オブジェクト, リテラル, 覚える, いい, コード, ...","[{""id"":235230.0,""similarity"":0.9764700532},{""i..."
5,5,25,763ccaa751ef0382c2bb,【Bootstrap】navbarの文字色を変えた時にちょっと詰まったこと,2019-01-31 00:35:07,5,bootstrap4,https://qiita.com/sayuprc/items/763ccaa751ef03...,"調べる, いい, 記事, 出る, くる, 詰まる, メモ, 残す, おく, 変更, 文字, ...","[{""id"":237035.0,""similarity"":0.9209199548},{""i..."
6,6,24,0e1c74e16e443cdfa1dd,IBM Cloud Developer Tools - CLI installer scri...,2019-01-31 00:38:41,0,"ibmcloud,poweshell",https://qiita.com/mori_dai/items/0e1c74e16e443...,"経緯, IBM Cloud, く, 取る, やく, 使う, の, 実行, 結果, エラーログ...","[{""id"":107453.0,""similarity"":0.8413683176},{""i..."
7,7,23,2be73956b788c745048f,【初心者向け】非同期にCRUDするReactアプリケーションを作る,2019-01-31 00:38:48,43,"JavaScript,json-server,React",https://qiita.com/t12u/items/2be73956b788c745048f,"Re・Act, 触る, 始める, プログラミング, 初心者, 非同期, データ, 扱う, ア...","[{""id"":234285.0,""similarity"":0.7544459105},{""i..."
8,8,22,9cfc5e003935001b919f,初めてのJavaScript④,2019-01-31 00:44:12,0,#初心者,https://qiita.com/prn_itg/items/9cfc5e00393500...,"オブジェクト, 配列, 複数, データ, まとめる, 管理, する, の, 用いる, 配列,...","[{""id"":234712.0,""similarity"":0.9967730641},{""i..."
9,9,21,f3d3db0819834a5dd596,【Python】prompt-toolkitのtext_before_cursorとget_...,2019-01-31 00:44:15,2,"Python,prompt_toolkit",https://qiita.com/tamago324/items/f3d3db081983...,"使う, いる, 自力, 調べる, みる, わかる, ISSUES, 質問, する, みる, ...","[{""id"":234173.0,""similarity"":0.9870455861},{""i..."


In [0]:
from gensim import similarities
from gensim import corpora, models
import pandas as pd

lda_path = "data/lda/lda.model"
corpus_path = "data/lda/cop.mm"


class Similar:
    def __init__(self, df, lda_model=None, corpus=None, recommend_num=15, debug=True):
        """
        コンストラクタ
        :param df: corpusの作成に使用したDataFrame(corpusに対応している)
        :param lda_model: 学習済みLDAモデル
        :param corpus: corpus
        :param recommend_num: 一記事における推薦記事数
        :param debug: False推奨
        """
        self.df = df
        self.lda_model = lda_model if lda_model else models.ldamodel.LdaModel.load(lda_path)
        self.corpus = corpus if corpus else corpora.MmCorpus(corpus_path)
        self.recommend_num = recommend_num
        self.debug = debug
        self.doc_index = similarities.docsim.MatrixSimilarity(self.lda_model[self.corpus])

    def basic_info(self, num):
        """
        dfのnum番目の情報を表示する
        :param num: 表示したいdfの番号
        :return:
        """
        if num > len(self.df):
            print("error")
            return
        print(self.df['url'][num])
        print(self.df['title'][num])
        print(self.df['tags_str'][num])
        print('like count: ' + str(self.df['likes_count'][num]))

    def similar_df(self, index):
        """
        推薦記事をDataFrameで返す
        DataFrameのcolumnは[index, similarity]
        :param index: index番目の記事に基づいて推薦する
        :return: DataFrame 推薦記事
        """
        # drop_list = ["sharing", "Unnamed", "created_at"]
        drop_list = ["sharing", "created_at", "article_id", "likes_count", "tags_str", "title", "url", "Unnamed: 0"]
        vec_lda = self.lda_model[self.corpus[index]]  # 50zigenn
        s = self.doc_index.__getitem__(vec_lda)  # 記事の数
        s = sorted(enumerate(s), key=lambda t: t[1], reverse=True)  # 並び替え（ヤバい）
        df_json = pd.DataFrame()
        for doc_id, sim in s[1:self.recommend_num + 1]:
            if self.debug:
                print("similarity: " + str(sim))
                self.basic_info(doc_id)
                print("===========")
            # df_json = pd.concat([df_json, df.loc[doc_id]], axis=0, sort=True)
            series = self.df.loc[doc_id].copy()
            series["similarity"] = sim
            df_json = df_json.append(series)

        df_json_ed = df_json.drop(drop_list, axis=1)
        return df_json_ed

    def similar_json(self, index):
        """
        推薦記事をjson型に変換する
        :param index: index番目の記事に基づいて推薦する
        :return:
        """
        df_json = self.similar_df(index)
        return df_json.to_json(orient='records')

    def append_json_to_df(self, df_num=None):
        """
        元のDataFrameにjsonの列を追加する
        問題点：sortにかかる計算量が N^2 log(N) (Nは記事数)
                一月分でも結構時間がかかる
        :param df_num:とりあえずdf_num番目まで表示 debug用
        :return:
        """
        df_num = df_num if df_num else len(self.df)
        json_list = []
        df_j = self.df[:df_num]
        for i in range(df_num):
            json_list.append(self.similar_json(i))
        if self.debug:
            print(len(json_list))
        df_j["json"] = json_list
        return df_j


In [0]:
simi = Similar(df, debug=False)

In [139]:
simi.append_json_to_df(df_num=20)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Unnamed: 0,id,article_id,title,created_at,likes_count,tags_str,url,sharing,json
0,0,30,b9c5eb649ada9224e928,CloudFunctionsからCloudStorageへ画像をアップロードする,2019-01-31 00:01:26,8,"CloudStorage,Firebase,FirebaseCloudFunctions",https://qiita.com/shim0mura/items/b9c5eb649ada...,"やる, てる, の, cloudfunctions, ある, ファイル, 画像, 読み込める...","[{""id"":107627.0,""similarity"":0.8924355507},{""i..."
1,1,29,06be5a8075e0c7924dbf,第二種スターリング数の指数型母関数表示とその応用,2019-01-31 00:03:56,1,"数学,組み合わせ",https://qiita.com/kaityo256/items/06be5a8075e0...,"スターリング数, stirling, Number, ご存知, 組み合わせ, 問題, 顔, ...","[{""id"":234315.0,""similarity"":0.9076671004},{""i..."
2,2,28,5789ee7a724c6cb786a9,Oracle Cloud(OCI)のcliのinput/outputで使うjsonに慣れる,2019-01-31 00:09:57,3,"JSON,cli,oci,oraclecloud",https://qiita.com/feifo/items/5789ee7a724c6cb7...,"新規作成, tips, 追加, Oracle Cloud, infrastructure, ...","[{""id"":109339.0,""similarity"":0.9266477823},{""i..."
3,3,27,e9ff4d797a19c84231a1,[.NET] コードを見直したくなる「参照型」等価判定の思わぬ落とし穴（特殊編）,2019-01-31 00:10:46,1,"VB.Net,C#,.NET,tips,等価演算子",https://qiita.com/CodeOne/items/e9ff4d797a19c8...,"続く, 特殊, インターン, プール, 文字列, リテラル, インターン, プール, テーブ...","[{""id"":236213.0,""similarity"":0.9793128967},{""i..."
4,4,26,a31c2405a4a32d43bb24,【テンプレートリテラル】と【オブジェクトリテラル】を覚えたら「ちょっと」いい感じにコードが書けた！,2019-01-31 00:14:52,30,"JavaScript,es6",https://qiita.com/micropig3402/items/a31c2405a...,"概要, テンプレート, リテラル, オブジェクト, リテラル, 覚える, いい, コード, ...","[{""id"":234712.0,""similarity"":0.908264637},{""id..."
5,5,25,763ccaa751ef0382c2bb,【Bootstrap】navbarの文字色を変えた時にちょっと詰まったこと,2019-01-31 00:35:07,5,bootstrap4,https://qiita.com/sayuprc/items/763ccaa751ef03...,"調べる, いい, 記事, 出る, くる, 詰まる, メモ, 残す, おく, 変更, 文字, ...","[{""id"":108312.0,""similarity"":0.9454466105},{""i..."
6,6,24,0e1c74e16e443cdfa1dd,IBM Cloud Developer Tools - CLI installer scri...,2019-01-31 00:38:41,0,"ibmcloud,poweshell",https://qiita.com/mori_dai/items/0e1c74e16e443...,"経緯, IBM Cloud, く, 取る, やく, 使う, の, 実行, 結果, エラーログ...","[{""id"":107664.0,""similarity"":0.8950814605},{""i..."
7,7,23,2be73956b788c745048f,【初心者向け】非同期にCRUDするReactアプリケーションを作る,2019-01-31 00:38:48,43,"JavaScript,json-server,React",https://qiita.com/t12u/items/2be73956b788c745048f,"Re・Act, 触る, 始める, プログラミング, 初心者, 非同期, データ, 扱う, ア...","[{""id"":107106.0,""similarity"":0.8116985559},{""i..."
8,8,22,9cfc5e003935001b919f,初めてのJavaScript④,2019-01-31 00:44:12,0,#初心者,https://qiita.com/prn_itg/items/9cfc5e00393500...,"オブジェクト, 配列, 複数, データ, まとめる, 管理, する, の, 用いる, 配列,...","[{""id"":236135.0,""similarity"":0.9567135572},{""i..."
9,9,21,f3d3db0819834a5dd596,【Python】prompt-toolkitのtext_before_cursorとget_...,2019-01-31 00:44:15,2,"Python,prompt_toolkit",https://qiita.com/tamago324/items/f3d3db081983...,"使う, いる, 自力, 調べる, みる, わかる, ISSUES, 質問, する, みる, ...","[{""id"":105392.0,""similarity"":0.8289510012},{""i..."
